***Participantes (RM - NOME):***<br>
RM339661 - David Claro<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>

###**Criar um classificador de sentimento aplicando técnicas de PLN**
---

Utilizando o dataset de revisões de filmes em português [1], criar um classificador de sentimentos que consiga um score na métrica F1 Score superior a 70%.

Devem utilizar uma amostra de 20% e randon_state igual a 42 para testar as implementações e mensurar a métrica F1 Score (usar o parâmetro average = 'weighted') o restante dos dados devem ser utilizados para o treinamento (80%).

Fique a vontade para testar os métodos de pré-processamento, abordagens, algoritmos e bibliotecas, mas explique e justifique suas decisões.
O trabalho poderá ser feito em grupo de até 4 pessoas (mesmo grupo do Startup One).

Separe a implementação do seu modelo campeão junto com a parte de validação/teste de forma que o professor consiga executar todo o pipeline do modelo campeão.

Composição da nota:
- 50% - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, etc.)
- 50% - Baseado na performance obtida com o dataset de teste (conforme recomendação da amostra) no seu modelo campeão e na validação que o professor processar (Métrica F1 Score).

[1] - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv

In [1]:
# CARREGANDO O DATA FRAME
import pandas as pd
df = pd.read_csv('https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv', encoding='utf-8')
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# Façam o download do arquivo e utilizem localmente durante os testes

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44514 entries, 0 to 44513
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   codigo      44514 non-null  int64 
 1   texto       44514 non-null  object
 2   sentimento  44514 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


Bom desenvolvimento!

In [3]:
#!pip install scipy==1.2.0
#!pip install gensim==2.0.0
!pip install spacy
!python -m spacy download pt

     |████████████████████████████████| 21.2MB 1.3MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp37-none-any.whl size=21186282 sha256=43a20dbfbd53e41a58b22c56b39c070a8c74ec25aed83a5959e7597d2b978680
  Stored in directory: /tmp/pip-ephem-wheel-cache-tb4k0xf5/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [4]:
import nltk
import spacy

import pandas as pd
import re
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB

In [5]:
# Carrega StopWords
nlp = spacy.load('pt')
nltk.download('stopwords')

# stopwords NLTK
stops_nltk = nltk.corpus.stopwords.words('portuguese')

# stopwords SpaCy
stops_spacy = nlp.Defaults.stop_words

# stopwords do SpaCy e NLTK combinadas
stops = list(set(stops_spacy).union(set(stops_nltk)))
len(stops)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


499

In [6]:
# função de lematização completa do documento


#def fn_lematiza_texto(texto):
#    sent = []
#    doc = nlp(texto)
#    for word in doc:
#        sent.append(word.lemma_)
#
#    return ' '.join(sent)

# função de lematização dos verbos do documento
#def fn_lematiza_verb_texto(texto):
#    sent = []
#    doc = nlp(texto)
#    for word in doc:
#        if word.pos_ =='VERB':
#            sent.append(word.lemma_)
#        else:
#            sent.append(word.text)
#    return " ".join(sent)


# função para limpar documento
def fn_limpa_texto(texto):
    texto = re.sub(r'@\w+','',texto)
    texto = re.sub(r'#','',texto)
    texto = re.sub(r'RT[\s]+','',texto)  
    #texto = re.sub(r'rt[\s]+','',texto)  
    texto = re.sub(r'https:/\/\S+','',texto)
    texto = re.sub(r'[,.:;]','',texto)
    texto = re.sub(r'\n',' ',texto)
    texto = re.sub(u'[^a-zA-ZâêîôûÂÊÎÔÛáéíóúÁÉÍÓÚàèìòùÀÈÌÒÙãõÃÕçÇ ]', '', texto)
    texto = texto.strip().lower()
    
    return texto

In [7]:
# aplica normalização no dataframe
df['texto_trat'] = df.texto.apply(fn_limpa_texto)

# aplica a lematização no dataframe
#df['texto_lemma'] = df.texto.apply(fn_lematiza_texto)

# aplica a lematização dos verbos no dataframe
#df['texto_lemma_verb'] = df.texto.apply(fn_lematiza_verb_texto)



In [8]:
# Contagem de termos simples
#vetor = CountVectorizer(ngram_range=(1,1))
#vetor = CountVectorizer(ngram_range=(1,2)) #0.7801952252146301 - Contagem simples, texto tratado, uni e bi gramas, regressão
#vetor = CountVectorizer(ngram_range=(1,1), stop_words=stops)
#vetor = CountVectorizer(ngram_range=(1,1), stop_words=stops_spacy)
#vetor = CountVectorizer(ngram_range=(1,2), stop_words=stops_nltk)

# Frequência de termos (TF)
#vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=False, norm='l1') #
## vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=False) #
#vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=False, stop_words=stops) #
#vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=False, stop_words=stops_nltk) #

# Frequência de termos - term frequency–inverse document frequency (TF-IDF)
#vetor = TfidfVectorizer(ngram_range=(1,1), use_idf=True, norm='l1') #
#vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=True, norm='l1') #
#vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=True) #0.7729036810537457 - TF-IDF, texto lematixado, uni e bi brama, Naive
vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=True, stop_words=stops) #
#vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=True, stop_words=stops_nltk) # lematixado

# texto tratado

vetor.fit(df.texto_trat)
text_vect = vetor.transform(df.texto_trat)

In [12]:
# separa as amostras de Treino (80%) e Texte (20%)
X_train,X_test,y_train,y_test = train_test_split(
        text_vect, 
        df['sentimento'], 
        test_size = 0.2,
        random_state = 42
    )

# define e treina o modelo de classificação
#modelo = LogisticRegression(random_state=42)
#modelo = DecisionTreeClassifier(random_state=42)
modelo = MultinomialNB()

modelo = LogisticRegression(penalty = 'l2', C = 100, random_state=42)

modelo.fit(X_train, y_train)

# valida a parformance
y_prediction = modelo.predict(X_test)
accuracy = accuracy_score(y_prediction, y_test)

print(text_vect.shape)
print(accuracy)

(44514, 2891076)
0.8955408289340672


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [13]:

f1_score(y_test, y_prediction, average='weighted')*100


89.55297580083781

####**Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.

In [11]:
# Modelo final

# vetorização tf-idf com combinação de unigrama e bigrama
vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=True) # 0.7729036810537457

# texto tratado e verbos lematizado
vetor.fit(df.texto_trat)#texto_lemma_verb
text_vect = vetor.transform(df.texto_trat)

# separa as amostras de Treino (80%) e Texte (20%)
X_train,X_test,y_train,y_test = train_test_split(
        text_vect, 
        df['sentimento'], 
        test_size = 0.2,
        random_state = 42
    )

# define e treina o modelo de classificação
modelo = MultinomialNB()

modelo.fit(X_train, y_train)

# valida a parformance
y_prediction = modelo.predict(X_test)
accuracy = accuracy_score(y_prediction, y_test)

print(text_vect.shape)
print(accuracy)

KeyboardInterrupt: ignored